**DEB**

**ANALYSE DU PROBLEME**
---

Problème de classification (28 classes) avec évaluation "sur mesure" (composite) qui tient compte de la performance et de l'équité du modèle<br><br>. 

Jeu de donnée de 25.000 personnes, après voir lancé la baseline, j'ai entrainé un **adversarial NN**, qui permettait de gagner un peu d'équité (passage de 80% à 85% sur le TRP gap) mais a entrainé un éffondrement de l'accuracy (de 65% à 35%). après avoir cherché à personaliser la fonction de perte (avec difficulté pour entrainer le modele), je suis repassée à une approche plus progressive, sur la régression logistique

**regression logisitique (baseline) => 3 problemes**<br>
1. ***l'échantillonage*** : Le "train_test_split" ne tient pas compte de S, nous ne savons pas si l'attribut protégé (H/F) est bien réparti entre X_train et X_test, cela n'est pas optimum pour l'apprentissage car certaines classes contiennent peu de points. Il peut aussi etre plus pertinent de fixer les hyperparametre grace à X_train, et d'entrainer le modele sur tout l'échantillon (avec les memes hyperparametres) avant de prédire X_test_true et de soumettre<br>
=> nous réaliserons des train_test_split en tenant compte de X et Y grace à Y56 = Y + 28*S <br>
=> nous ferons des soumissions double (modele appris sur X_train et aussi X) pour le datachallenge, pour améliorer les qualités prédicitive du modèles <br><br>
2. la fonction d'évaluation est une ***moyenne de scores calculé par classes***, chaque classe a donc le meme poids dans le score final. Or, l'apprentissage est fait pour optimser l'accuracy de l'échantillon (chaque élément à le meme poids). Comme la distribution de l'échatillon est très mal équilibrée entre les classes (93 à 8.285 personnes), le modèle ne peut apprendre correctement<br>
=> nous essaierons pour la forme del a data augmentation, mais sans conviction compte tenu de l'embedding sémantique fait avec BERT<br>
=> nous utiliserons une correction de la fonction de pertes pour y inclure le poids de chaque classe (Y et Y56)<br>
=> nous essaierons du contrastive learning pour obtenir un espace de réprésentation adéquat<br><br>
2. ***l'apprentissage n'est pas optimal (alignement et hyperparametres à améliorer)***. Il manque la régularization et du cross validation. L'apprentissage ,'est pas aligné avec "score final": le modele fait pour optimiser l'accuracy (la précision), or la fonction d'évaluation est une moyenne entre le F1 score (moyenne harmonique de la précision et du rappel) et une métrique de fairness. Le F1 score pénalise les écarts importants entre la précision et le rappel. Le score<br>
=> nous adapterons le fonctions de pertes et/ou l'architecture des modèles intégrer au mieux notrescore final
=> nous ajouterons systematiquement des termes de régularisations et de cross validation (stratified k-fold cross validation si possibl)<br> 

**METHODOLOGIE**
---

nous **ajusterons la fonction de pertes**, pour qu'elle tienne compte au mieux de la fonction d'évaluation, qui est composée de 2 éléments : le F1 score et le 1-equal ooprtunity gap<br>
- le 'macro f score' (F1 score) n'est pas dérivable (à valider), nous l'approcherons donc par accuracy et le recall<br>
- true positive gap : 1 - TRP_GAP est calculé comme 
A noter que le F1 score n'est pas dérivable. However, we can approximate the F1 score in a differentiable manner by using the ***Sørensen–Dice coefficient***, which is closely related to the F1 score and is differentiable. The Sørensen–Dice coefficient is defined as \(2 * \frac{precision * recall}{precision + recall}\), which is equivalent to the F1 score formula. For a differentiable approximation, we can use soft versions of precision and recall.
<br><br>



**A) Import, loading data, eval functions**
---

In [1]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import normalize
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_val_split, StratifiedKFold, cross_val_score
from sklearn.datasets import make_classification
from sklearn.metrics import make_scorer #, f1_score

#from Data_challenge_fairness_2024.evaluator import *
from evaluator import *


In [2]:

path_model = 'BIAS_models/'  # must finish by '/'
path_Y_pred_true = 'BIAS_Y_pred_true/'  # must finish by '/'

# with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)
# with open(path_model + name + '.pkl', 'rb') as f: model = pickle.load(f)

def get_final_score(Y_pred, Y, S):
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2
    return final_score
    
    
def get_scores(Y_pred, Y, S, will_print=1):
    accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
    if will_print==1 : print(f"Accuracy on transformed test data: {accuracy}")
    
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2
    macro_f1 = eval_scores['macro_fscore']
    inv_macro_gap = 1-eval_scores['TPR_GAP']

    if will_print==1: #print results
        print('final score :',final_score)
        print('macro_f1    :',eval_scores['macro_fscore'])
        print('inv_macro_gap',1-eval_scores['TPR_GAP'])
    
    return accuracy, final_score, macro_f1, inv_macro_gap

In [3]:
#####################################################
# Load pickle file and convert to numpy array
#####################################################

with open('data-challenge-student.pickle', 'rb') as handle:
    # dat = pickle.load(handle)
    dat = pd.read_pickle(handle)

#Check keys()
print(dat.keys())
X = dat['X_train']
Y = dat['Y']
S = dat['S_train']

X_test = dat['X_test']
S_test = dat['S_test']

Y56= Y + 28*S
#X, X_test,Y,S, S_test = dat[1]

print(X.shape,Y.shape,S.shape,X_test.shape,S_test.shape)

X_train, X_val, Y_train, Y_val, S_train, S_val = train_test_split(X, Y, S, test_size=0.3, random_state=42)

path_model_ = 'BIAS_models/'
path_Y_pred = 'BIAS_Y_pred/'

dict_keys(['X_train', 'X_test', 'Y', 'S_train', 'S_test'])
(27749, 768) (27749,) (27749,) (11893, 768) (11893,)


**MODEL 6 - ADPAPTING LOSS FUNCTION IN LOGISTIC REGRESSION (PYTORCH)**
---
1. Functions for Custom loss function (re-written in pytorch)
2. Functions for Stratified K-fold and training batches
3. Running Model 6
4. Optimising Model 6 hyperparameters

In [ ]:
''' PROBLEME SUR FINAL SCORE QUI DEPEND DE S, OR SICKIT LEARN NE GERE
PAS S DANS LA CROSS VALIDATION, LA LOSS FUNCTION PERSONALISEE NE PEUT 
ETRE CALCULEE  =======> CODAGE SUR PYTORCH

##########################################################################
# 5. CUSTOM LOSS FUNCTION (Using stratified K + custom_scorer)
# + Debiasing with orthogonal projection + Optimised Logisitic Regression
##########################################################################

name ='5_Reglog_StratKFold1_Orthogonal_CustomLoss' # changer clf_i

#---------------------------------------------------------------------
# Debiasing X (Projection of X orthogonally to bias) 
#---------------------------------------------------------------------

# projection in new representation space
debiased_X = remove_bias(X, bias)

#------------------------------------------------------------------------
# CREATION OF CUSTOM LOSS FUNCTION (FINAL SCORE) 
# -----------------------------------------------------------------------

#It is not possible to split S with the stratified k-fold implementation 
# in sickit learn. We will use Y56 = Y + 28*S to train logistic model
# and derive from Y56 the value of Y = Y56 % 28 and S = Y56//28

def final_score_S(Y,Y_pred):
    # wrapper function to include S
    # Note order : custom scorer expects (y_true, y_pred) as inputs
    
    return get_final_score(Y_pred,Y,S)

custom_scorer = make_scorer(final_score_S)

#---------------------------------------------------------------------
# TRAINING MODEL AND PREDICTING
#---------------------------------------------------------------------

clf_5 = LogisticRegression(random_state=0, max_iter=5000,verbose=0,penalty='l2', C=0.2, tol=0.0001)

# Préparer la procédure de validation croisée k-fold stratifiée
cv = StratifiedKFold(n_splits=1)
scores = cross_val_score(clf_3, debiased_X, Y, scoring=custom_scorer ,cv=cv, n_jobs=-1)
clf_5.fit(debiased_X, Y56)
model = clf_5
# with open(path_model + name + '.pkl', 'rb') as f: clf_5 = pickle.load(f)

# predicting and assessing
Y56_pred = model.predict(X_test)
#S_pred = Y56//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
Y_pred = Y56_pred % 28  # reste (original Y)   ex 33% 28 = classe 5
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_pred,Y_test,S_test)

# predict X_test_true and save
modified_X_test_true = remove_info(X_test_true, bias)  # debiasing
Y56_pred_true = model.predict(modified_X_test_true)
Y_pred_true = Y56_pred_true % 28  # reste (original Y)   ex 33% 28 = classe 5
results=pd.DataFrame(Y_pred_true, columns= ['score'])
results.to_csv(path_Y_pred_true + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)'''

In [40]:
##########################################################################
# 5. CUSTOM LOSS FUNCTION WITH 56 CLASS CLASSIFIER (USING Y56 = Y + S*28)
# + Debiasing with orthogonal projection + Optimised Logisitic Regression
##########################################################################

name ='5_Reglog56_StratKFold_Orthogonal_CustomLoss' # changer clf_i

#---------------------------------------------------------------------
# Debiasing X (Projection of X orthogonally to bias) 
#---------------------------------------------------------------------

# projection in new representation space
debiased_X = remove_bias(X, bias)

#------------------------------------------------------------------------
# CREATION OF CUSTOM LOSS FUNCTION (FINAL SCORE) 
# -----------------------------------------------------------------------

#It is not possible to split S with the stratified k-fold implementation 
# in sickit learn. We will use Y56 = Y + 28*S to train logistic model
# and derive from Y56 the value of Y = Y56 % 28 and S = Y56//28

def final_score_S(Y56,Y_pred):
    '''custom scorer expects (y_true, y_pred) as inputs
    Inputs : Y56 and Y pred
    Outputs : final score on 28 classes with Y56 trick (S=Y56//28 and Y=Y56%28)'''
    S = Y56//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
    Y = Y56 % 28  # reste (original Y)   ex 33% 28 = classe 5 
    return get_final_score(Y_pred,Y,S)

custom_scorer = make_scorer(final_score_S)

#---------------------------------------------------------------------
# TRAINING MODEL AND PREDICTING
#---------------------------------------------------------------------

'''BlockingIOErrorclf_5 = LogisticRegression(random_state=42, max_iter=5000,verbose=0,penalty='l2', C=0.2, tol=0.0001)

# Préparer la procédure de validation croisée k-fold stratifiée
cv = StratifiedKFold(n_splits=10)
scores = cross_val_score(clf_5, debiased_X, Y56, scoring=custom_scorer ,cv=cv, n_jobs=-1)
clf_5.fit(debiased_X, Y56)'''
with open(path_model + name + '.pkl', 'rb') as f: clf_5 = pickle.load(f)
model = clf_5

# predicting and assessing
Y56_pred = model.predict(X_val)
#S_pred = Y56//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)
Y_pred = Y56_pred % 28  # reste (original Y)   ex 33% 28 = classe 5
accuracy, final_score, macro_f1, inv_macro_gap = get_scores(Y_val,Y_val,S_val)


# predict X_test_true and save
modified_X_test = remove_bias(X_test, bias)  # debiasing
Y56_pred = model.predict(modified_X_test)
Y_pred = Y56_pred % 28  # reste (original Y)   ex 33% 28 = classe 5
results=pd.DataFrame(Y_pred, columns= ['score'])
#results.to_csv(path_Y_pred + "Data_Challenge_" + name + ".csv", header = None, index = None)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)

Accuracy on transformed test data: 1.0
final score : 0.9999995472208593
macro_f1    : 1.0
inv_macro_gap 0.9999990944417185


In [ ]:
##########################################################################
# 6. CUSTOM LOSS FUNCTION IN PYTORCH (CLASSIFIER ON 28 ORIGINAL CLASSES)
# + Debiasing with orthogonal projection + Optimised Logisitic Regression
##########################################################################

name ='5_Reglog56_StratKFold_Orthogonal_CustomLoss' # changer clf_i

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader


#########################################################################
#     CUSTOM LOSS FUNCTION AND EVALUATION FUNCTIONS
#         (RE-WRITTEN FOR PYTORCH)
#
#   soft_f1_loss
#   macro_soft_f1_loss
#   get_macro_f1
#   get_tpr_gap
#   get_macro_tpr_gap
#   soft_final_score_loss
#   get_final_score
#
###########################################################################


def soft_macro_f1_loss(y_true, y_pred):
    """
    Differentiable approximation of the macro F1 score as a loss function.
    Calculates the F1 score for each class independently and then takes the average.
    Inputs :
        y_true must be one hot encoded
    """
    y_pred_one_hot = torch.nn.functional.one_hot(y_pred, num_classes=Y_train.nunique()) if len(y_pred.shape) == 1 else y_pred
    #y_pred_probs = torch.softmax(y_pred_one_hot, dim=1)
    
    tp = torch.sum(y_true * y_pred, dim=0)
    pp = torch.sum(y_pred, dim=0)
    ap = torch.sum(y_true, dim=0)
    
    precision = tp / (pp + 1e-6)
    recall = tp / (ap + 1e-6)
    
    f1_per_class = 2 * (precision * recall) / (precision + recall + 1e-6)
    macro_f1 = torch.mean(f1_per_class)   # Mean to aggregate over all classes
    
    loss = 1 - macro_f1  # Minimizing loss is maximizing macro F1 score
    return loss


def get_macro_f1(y_true, y_pred):
    """
    Calculate the exact macro F1 score for evaluation.
    Input : any format as tensors will be converted to Tensors of true label if dim >1 . Can be :
        - Tensor of probabilities(y_pred_probs) dimension (n,28)
        - Tensor of labels, one hote encoded (y_pred_one_hot) dimension (n,28)
        - Tensor of labels (y_pred_tensor) dimension (n,1)
    Ouput : scalar
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true

    " predict macro f1"
    f1 = f1_score(y_true_labels.cpu().numpy(), y_pred_labels.cpu().numpy(), average='macro')
    return f1

def get_tpr_gap(y_true, y_pred, protected_attribute, class_idx):
    """
    Calculate the TPR gap for a specific class across protected groups.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred_probs: Tensor of predicted probabilities (after softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    - class_idx: Index of the class for which to calculate the TPR gap.
    
    Returns:
    - TPR gap for the specified class.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true
    
    # Calculate overall TPR for the current class
    overall_mask = y_true_labels == class_idx
    overall_tpr = torch.sum((y_pred_labels == class_idx) & overall_mask).float() / (torch.sum(overall_mask).float() + 1e-6)
    
    # Initialize list to store TPR for each protected group
    group_tprs = []
    
    # Calculate TPR for each protected group
    for group_val in protected_attribute.unique():
        group_mask = (protected_attribute == group_val) & overall_mask
        group_tpr = torch.sum((y_pred_labels == class_idx) & group_mask).float() / (torch.sum(group_mask).float() + 1e-6)
        group_tprs.append(group_tpr)
    
    # Calculate TPR gap for the current class
    tpr_gaps = torch.abs(torch.tensor(group_tprs) - overall_tpr)
    
    return torch.mean(tpr_gaps)  # Return the mean TPR gap for this class

def get_macro_tpr_gap(y_true, y_pred, protected_attribute):
    """
    Calculate the average TPR gap per class by calling tpr_gap for each class.
    
    Args:
    - y_true: Tensor of true labels, one-hot encoded.
    - y_pred: Tensor of predicted logits (before softmax).
    - protected_attribute: Tensor indicating group membership for each instance.
    
    Returns:
    - Average TPR gap across all classes.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true
    
    # Initialize list to store TPR gaps for all classes
    class_tpr_gaps = []
    
    # Iterate over each class
    num_classes = len(y_true_labels.unique())
    for class_idx in range(num_classes):
        class_tpr_gap = get_tpr_gap(y_true_labels, y_pred_labels, protected_attribute, class_idx)
        class_tpr_gaps.append(class_tpr_gap)
    
    # Calculate the average TPR gap across all classes
    avg_tpr_gap = torch.mean(torch.stack(class_tpr_gaps))
    
    return avg_tpr_gap


def soft_final_score_loss(y_true, y_pred, protected_attribute):
    """
    Combine soft macro F1 score and TPR gap to create a final evaluation metric.
    """
    soft_macro_f1 = soft_macro_f1_loss(y_true, y_pred)  # Calculate soft macro F1 score
    macro_tpr_gap = get_macro_tpr_gap(y_true, y_pred, protected_attribute)  # Calculate TPR gap
    
    soft_final_score = ( soft_macro_f1 + (1 - macro_tpr_gap) ) / 2
    return soft_final_score

def get_final_score(y_true, y_pred, protected_attribute):
    """
    Combine soft macro F1 score and TPR gap to create a final evaluation metric.
    """
    #convert Tensors to 1 dimension (labels ranging from 0 to 27) if necessary
    y_pred_labels = torch.argmax(y_pred, dim=1) if y_pred.ndim > 1 else y_pred
    y_true_labels = torch.argmax(y_true, dim=1) if y_true.ndim > 1 else y_true

    macro_f1 = get_macro_f1(y_true_labels, y_pred_labels)  # Calculate macro F1 score
    macro_tpr_gap = get_macro_tpr_gap(y_true_labels, y_pred_labels, protected_attribute)  # Calculate macro TPR gap
    
    final_score = (macro_f1 + (1 - macro_tpr_gap)) / 2
    return final_score

#---------------------------------------------------------------------
# TORCH EVALUATION FUNCTIONS
#--------------------------------------------------------------------


def evaluate(Y_pred,Y,S,will_print=1):
    '''returns model accuracy, final score, macro fscore ans TPR gap
    input : 2 np arrays of same dimension
    output : array of 4 values
    '''
    accuracy= accuracy_score(Y, Y_pred)  # Y_test are your original test labels
    print(f"Accuracy on transformed test data: {accuracy}")
    eval_scores, confusion_matrices_eval = gap_eval_scores(Y_pred, Y, S, metrics=['TPR'])
    final_score = (eval_scores['macro_fscore']+ (1-eval_scores['TPR_GAP']))/2

    if will_print==1:
        #print results
        print('final score',final_score)
        print('macro_fscore',eval_scores['macro_fscore'])
        print('1-eval_scores[\'TPR_GAP\']',1-eval_scores['TPR_GAP'])
    
    return accuracy, final_score, eval_scores['macro_fscore'],1-eval_scores['TPR_GAP'] , eval_scores , confusion_matrices_eval

# to predict X_test and save to file

def save_Y_pred_tofile(X, model, name): # adapted to torch
    
    # save probabilities for each Xi (dim=28)
    y_pred_probs = model(X)
    probs=pd.DataFrame(y_pred_probs.detach().numpy(), columns= list(range(0,28)))
    file_name_probs = "y_pred_probs/y_pred_probs_"+str(name)+".csv"
    probs.to_csv(file_name_probs, header = None, index = None)

    # save predicted labels for each Xi (dim=1)
    y_pred = torch.argmax(y_pred_probs, dim=1)
    results=pd.DataFrame(y_pred.numpy(), columns= ['score'])
    file_name = "y_pred/Data_Challenge_"+str(name)+".csv"
    results.to_csv(file_name, header = None, index = None)

    return y_pred, y_pred_probs
    

def print_cassif_report(Y_pred,Y_test):
    # Convert Y_pred to a DataFrame
    Y_pred_df = pd.DataFrame(Y_pred_tensor.numpy(), columns=['Predicted'])

    # Evaluate Y_pred compared to Y_test (assuming Y_test is a numpy array or a pandas Series)
    table = classification_report(Y_test, Y_pred_df['Predicted'])

    return table

#########################################################################
#     ENTRAINEMENT DU MODEL (AVEC MINI BATCH / SANS CROSS VALIDATION)
#########################################################################


def train_NN_with_custom_loss(model, optimizer, batch_size, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor):

    # 1. Convertir les tensors en datasets puis en DataLoader pour gérer les mini-batchs
    train_dataset = TensorDataset(X_train_tensor, Y_train_one_hot, S_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

    test_dataset = TensorDataset(X_test_tensor, Y_test_one_hot, S_test_tensor)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    
    # 2. Paramètres pour l'arrêt précoce
    # -----------------------------------
    patience = 5  # Nombre d'époques à attendre après la dernière amélioration de la perte de validation
    best_loss = None
    early_ending = None
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        
        model.train()
        train_loss = 0.0
        
        # 1/ exécuter les minibatches et recupérer la loss moyenne
        for X_batch, Y_batch, S_batch in train_loader:
            # Y_batch est one hot
            
            model.train()
            optimizer.zero_grad()
            outputs_train = model(X_batch)
            loss = soft_final_score_loss(Y_batch, outputs_train, S_batch)
            loss.backward()
            optimizer.step()

            # save mini-batch loss
            train_loss += loss.item()
        
        # Average loss pour l'epoch (après boucle mini-batchs)
        train_loss = train_loss / len(train_loader)       
        
        # 2. Vérifier si la perte de validation s'est améliorée (arret précoce)

        # Evaluation sur le jeu de données de test
        model.eval()
        test_loss = 0.0
        
        with torch.no_grad():
            for X_batch_test, Y_batch_test, S_batch_test in test_loader:
                outputs_test = model(X_batch_test)
                #Y_batch_test_one_hot = torch.nn.functional.one_hot(Y_batch_test, num_classes=Y_train.nunique())
                loss_test = soft_final_score_loss(Y_batch_test, outputs_test, S_batch_test)
                test_loss += loss_test.item()
                
        #average_test_loss = running_loss_test / len(test_loader)
        test_loss = test_loss / len(test_loader)
       
        # check if improvement in loss (compared to last epoch)
        if best_loss is None or test_loss < best_loss:
            best_loss = test_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
            if epochs_without_improvement >= patience:
                print(f'Arrêt précoce après {epoch+1} époques')
                early_ending = epoch + 1
                break  # Arrêter l'entraînement
        
        # 3. Impression de l'apprentissage et des scores train et test
        if epoch==0 or (epoch+1) % 10 == 0:
            
            with torch.no_grad():
                
                # Calculate metrics for training data
                outputs_train = model(X_train_tensor) # probabilities
                # Evaluate predictions on training data
                final_score_train_ = get_final_score(Y_train_tensor, outputs_train, S_train_tensor)
                macro_f1_train = get_macro_f1(Y_train_tensor, outputs_train)
                inv_macro_tpr_gap_train = 1 - get_macro_tpr_gap(Y_train_tensor, outputs_train, S_train_tensor)
            
                # Calculate metrics for test data
                outputs_test = model(X_test_tensor)
                # Evaluate predictions on training data
                final_score_test_ = get_final_score(Y_test_tensor, outputs_test, S_test_tensor)
                macro_f1_test = get_macro_f1(Y_test_tensor, outputs_test)
                inv_macro_tpr_gap_test = 1 - get_macro_tpr_gap(Y_test_tensor, outputs_test, S_test_tensor)

                print(f'Epoch {epoch+1}, Loss: {loss.item()}, Final Score Train: {final_score_train_.item()}, Final Score Test: {final_score_test_.item()} (gap {final_score_test_-final_score_train_}) macro F1 Train: {macro_f1_train}, macro F1 Test: {macro_f1_test}, 1-TPR Gap Train: {inv_macro_tpr_gap_train}, 1-TPR Gap Test: {inv_macro_tpr_gap_test}')
            
    # 4. Make Predictions and Evaluate with final_score
    # -------------------------------------------------
            
    with torch.no_grad():
        model.eval()

        Y_train_pred_probs = model(X_train_tensor) # dim = 28 (Probabilities for each class)
        
        # # Y_train_pred_tensor = torch.argmax(Y_train_pred_probs, dim=1)  # dim = 1 (Get the class with the highest probability)
        final_score_train = get_final_score(Y_train_tensor, Y_train_pred_probs, S_train_tensor)

        Y_pred_probs = model(X_test_tensor) # dim = 28 (Probabilities for each class)
        Y_pred_tensor = torch.argmax(Y_pred_probs, dim=1)  # dim = 1 (Get the class with the highest probability)
        macro_f1 = get_macro_f1(Y_test_tensor, Y_pred_tensor)
        inv_macro_tpr_gap = 1 - get_macro_tpr_gap(Y_test_tensor, Y_pred_probs, S_test_tensor)
        final_score = get_final_score(Y_test_tensor, Y_pred_probs, S_test_tensor)
        
        print(f'Final Evaluation Score: {final_score.item()} gap {final_score.item()-final_score_train.item()} || Macro F1: {macro_f1.item()} 1-TPR_gap: { inv_macro_tpr_gap.item() }')

    return model, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap, early_ending,final_score_train


In [ ]:
##############################################################
# train_test_split (np.arrays)
##############################################################

# Diviser les données en ensembles d'entraînement et de test
X_train, X_test, Y56_train, Y56_test = train_test_split(debiased_X, Y56, test_size=0.2, random_state=42)

Y_train = Y56_train % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_train = Y56_train//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)

Y_test = Y56_test % 28  # reste (original Y)   ex 33% 28 = classe 5 
S_test = Y56_test//28   # facteur (original S) ex 33//28 = 1 (attribut protégé)

# impression des dimensions
print('train:',X_train.shape,Y_train.shape,S_train.shape)
print('test:',X_test.shape,Y_test.shape, S_test.shape)

##############################################################
# 1. Transform DataFrames into Tensors
##############################################################

X_tensor = torch.tensor(X.values, dtype=torch.float32)
Y_tensor = torch.tensor(Y.values, dtype=torch.long)
S_tensor = torch.tensor(S.values, dtype=torch.long)

X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
Y_train_tensor = torch.tensor(Y_train.values, dtype=torch.long)
S_train_tensor = torch.tensor(S_train.values, dtype=torch.long)

X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32)
Y_test_tensor = torch.tensor(Y_test.values, dtype=torch.long)
S_test_tensor = torch.tensor(S_test.values, dtype=torch.long)

Y_train_one_hot = torch.nn.functional.one_hot(Y_train_tensor, num_classes=Y_train.nunique())
Y_test_one_hot = torch.nn.functional.one_hot(Y_test_tensor, num_classes=Y_train.nunique())

X_test_true_tensor = torch.tensor(X_test_true.values, dtype=torch.float32)
#S_test_true_tensor = torch.tensor(S_test_true.values, dtype=torch.long)

# impression des dimensions
print('train_tensor:',X_train_tensor.shape,Y_train_tensor.shape,S_train_tensor.shape, type(X_train_tensor))
print('test_tensor:',X_test_tensor.shape,Y_test_tensor.shape, S_test_tensor.shape, type(X_test_tensor))
print('Y_train_one_hot:',Y_train_one_hot.shape, type(Y_train_one_hot))
print('X_test_true_tensor:',X_test_true_tensor.shape, type(X_test_true_tensor))

In [ ]:
#################################################
#       TEST D'UN MODEL (ET DU CODE)
################################################


# 1. Define the model and optimizer and train
# --------------------------------------------------

model = nn.Sequential(
    nn.Linear(768, 28),  # Assuming 768 input features and 28 classes
    nn.ReLU(),  # Adding a ReLU activation function
    nn.Linear(28, 28),
    nn.Softmax(dim=1),  # LogSoftmax for multi-class classification
    )  

batch_size = 512
learning_rate=0.001
optimizer = optim.Adagrad(model.parameters(), lr=learning_rate, weight_decay=1/0.2)
num_epochs = 1000

# 2. Train the model with the custom loss function final_eval
# -----------------------------------------------------------
name = 'NN-28-28_Adam'+'_lr_'+str(learning_rate)+'_batch_size_'+str(batch_size)
print('\n\n Starting to train model', name)
model_trained, Y_pred_probs, Y_pred_tensor, final_score, macro_f1, inv_macro_tpr_gap, early_ending, final_score_train = train_NN_with_custom_loss(model,optim.Adam(model.parameters(), lr=learning_rate), batch_size, X_train_tensor, Y_train_tensor, S_train_tensor, X_test_tensor, Y_test_tensor, S_test_tensor)

# Res=pd.DataFrame(columns=['model','optimizer','lr','batch_size','early_ending', 'final_score_train','final_score','macro_f1','macro_tpr_gap'])
# Res.loc[i]=[name,optimizer,learning_rate,batch_size, early_ending,final_score_train, final_score, macro_f1, inv_macro_tpr_gap]

# Save predictions on X_test_true
save_Y_pred_tofile(X_test_true_tensor, model_trained,name)

# save model
with open(path_model + name + '.pkl', 'wb') as f: pickle.dump(model, f)